In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Imputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import category_encoders as cs
from sklearn.pipeline import FeatureUnion

In [28]:
trafficVolum = pd.read_csv("traffic_flow_data.csv",index_col=0)
#trafficVolum.describe()
target_data = trafficVolum.loc[:,"Segment23_(t+1)"] 
len(trafficVolum.columns)
#world cup attributes
w_features=trafficVolum.iloc[:,np.arange(449)].copy()
#world cup goal result
w_target=trafficVolum.iloc[:,449].copy()


449

In [31]:
# Create a class to select numerical or categorical columns 
# since Scikit-Learn doesn't handle DataFrames in this wise manner yet
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

def feature_process(features):

    w_features_num = features;

    num_pipeline = Pipeline([
            ('selector', DataFrameSelector(list(w_features_num))),
            ('imputer', Imputer(strategy="median")),
            ('std_scaler', StandardScaler(with_mean = False)),
        ])

    full_pipeline = num_pipeline
    return full_pipeline

In [32]:
full_pipeline = feature_process(w_features)

feature_prepared = pd.DataFrame(data=full_pipeline.fit_transform(w_features),index=np.arange(1,len(w_features) + 1))

train_data = feature_prepared[:int(0.8 * len(feature_prepared))]
test_data = feature_prepared[int(0.8 * len(feature_prepared)):]
train_results = w_target[:int(0.8 * len(w_target))]
test_results = w_target[int(0.8 * len(w_target)):]

In [37]:
# Decision Trees ?
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score


params = {'max_leaf_nodes': list(range(2, 5)), 'min_samples_split': [2,3,4,5]}
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1, verbose=1)
grid_search_cv.fit(train_data, train_results)
print grid_search_cv.best_estimator_

T_predict = grid_search_cv.predict(test_data)
print("The prediction accuracy using the decision tree is : {:.2f}%.".format(100*accuracy_score(test_results, T_predict)))



Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:   13.6s finished


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=4, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=42, splitter='best')
The prediction accuracy using the decision tree is : 0.87%.


In [41]:
#Logistic Regression Model
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter = 3)
model.fit(train_data,train_results)
T_predict = model.predict(test_data)
W_predict = model.predict(train_data)

print model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=3, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [46]:
from sklearn.metrics import mean_squared_error, r2_score


print(' ')
# The coefficients
#print('Coefficients and Intercept are: ', model.coef_,"   ",model.intercept_,' respectively')
# The mean squared error
print('_________________###################____________________')
print("Mean squared error for testing data: %.2f"
      % mean_squared_error(test_results, T_predict))
# Explained variance score: 1 is perfect prediction
print('Variance score for testing data: %.2f' % r2_score(test_results, T_predict))
print('******************************************************* ')
print("Mean squared error for training data: %.2f"
      % mean_squared_error(train_results, W_predict))
# Explained variance score: 1 is perfect prediction
print('Variance score for training data: %.2f' % r2_score(train_results, W_predict))

 
_________________###################____________________
Mean squared error for testing data: 78420.84
Variance score for testing data: -1.17
******************************************************* 
Mean squared error for training data: 73602.48
Variance score for training data: -0.83


In [47]:
# Linear Ridge Regression
from sklearn.linear_model import Ridge

model = Ridge(alpha=1.0)
model.fit(train_data, train_results)
T_predict = model.predict(test_data)
W_predict = model.predict(train_data)
print model

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)


In [48]:
print(' ')
# The coefficients
#print('Coefficients and Intercept are: ', model.coef_,"   ",model.intercept_,' respectively')
# The mean squared error
print('_________________###################____________________')
print("Mean squared error for testing data: %.2f"
      % mean_squared_error(test_results, T_predict))
# Explained variance score: 1 is perfect prediction
print('Variance score for testing data: %.2f' % r2_score(test_results, T_predict))
print('******************************************************* ')
print("Mean squared error for training data: %.2f"
      % mean_squared_error(train_results, W_predict))
# Explained variance score: 1 is perfect prediction
print('Variance score for training data: %.2f' % r2_score(train_results, W_predict))

 
_________________###################____________________
Mean squared error for testing data: 834.91
Variance score for testing data: 0.98
******************************************************* 
Mean squared error for training data: 1127.08
Variance score for training data: 0.97


In [50]:
# Linear Regression Model
from sklearn.linear_model import LinearRegression

model = LinearRegression(n_jobs = None)
model.fit(train_data, train_results)
T_predict = model.predict(test_data)
W_predict = model.predict(train_data)
print model

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)


In [51]:
print(' ')
# The coefficients
#print('Coefficients and Intercept are: ', model.coef_,"   ",model.intercept_,' respectively')
# The mean squared error
print('_________________###################____________________')
print("Mean squared error for testing data: %.2f"
      % mean_squared_error(test_results, T_predict))
# Explained variance score: 1 is perfect prediction
print('Variance score for testing data: %.2f' % r2_score(test_results, T_predict))
print('******************************************************* ')
print("Mean squared error for training data: %.2f"
      % mean_squared_error(train_results, W_predict))
# Explained variance score: 1 is perfect prediction
print('Variance score for training data: %.2f' % r2_score(train_results, W_predict))

 
_________________###################____________________
Mean squared error for testing data: 843.81
Variance score for testing data: 0.98
******************************************************* 
Mean squared error for training data: 1123.80
Variance score for training data: 0.97
